# Test 01

In [1]:
# load data
from fivedreg.data.load_data import load_data

data = load_data("../data/coursework_dataset.pkl")
print(data['X'].shape, data['y'].shape)

(5000, 5) (5000,)


In [4]:
# create model and trainer
from fivedreg.model.naive_nn import NaiveMLP
from fivedreg.trainer.nn_trainer import NNTrainer

mlp_config = {
    'hidden_dims': [128],
    'dropout': 0.0,
    'activation': 'relu',
}

training_config = {
    'learning_rate': 1e-3,
    'batch_size': 32,
    'epochs': 100,
    'early_stopping_patience': 0,
    'weight_decay': 0.0,
    'checkpoint_dir': './checkpoints',
}

trainer = NNTrainer(
    model_class=NaiveMLP,
    model_config=mlp_config,
    training_config=training_config,
    device='cpu',
)

Trainer initialized on device: cpu
Model: NaiveMLP(hidden_dims=[128], dropout=0.0, n_params=897)


In [5]:
results = trainer.train(data['X'], data['y'])

Epoch [10/100] Train Loss: 0.111767
Epoch [20/100] Train Loss: 0.057628
Epoch [30/100] Train Loss: 0.030957
Epoch [40/100] Train Loss: 0.019441
Epoch [50/100] Train Loss: 0.015131
Epoch [60/100] Train Loss: 0.013311
Epoch [70/100] Train Loss: 0.010856
Epoch [80/100] Train Loss: 0.009567
Epoch [90/100] Train Loss: 0.008483
Epoch [100/100] Train Loss: 0.007431

Training completed!
